# Import President data from XML file

## Standard imports

In [2]:
# %load imports.py
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import seaborn as sns
sns.set()
import lxml.etree as ET
from datetime import date


## Create empty dataframe with column and index names

In [3]:
df = pd.DataFrame(columns='first_name last_name dob dod bplace bstate tstart tend party'.split(), index=pd.Index(data=[], name='term')) 
df

,first_name,last_name,dob,dod,bplace,bstate,tstart,tend,party
term,,,,,,,,,


## Month lookup
A dictionary to translate the month name (e.g., 'Jan') into the integer month number

In [4]:
months = 'Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec'.split()
month_lookup = { month:num for num, month in enumerate(months, 1)}
month_lookup

{'Jan': 1,
 'Feb': 2,
 'Mar': 3,
 'Apr': 4,
 'May': 5,
 'Jun': 6,
 'Jul': 7,
 'Aug': 8,
 'Sep': 9,
 'Oct': 10,
 'Nov': 11,
 'Dec': 12}

## Function to find date values and make a Python date object
We use xpath (`.//TAG`) to find the date tag, then grab the text from the sub tags. The year and day must be converted to integers, and the month value is a string
that is looked up in the `month_lookup` dictionary.

In [5]:
def get_date(doc, tag):
    year = row.findtext(f'.//{tag}/year')
    if year == '':  # still alive
        return date.today()
    month = row.findtext(f'.//{tag}/month')
    day = row.findtext(f'.//{tag}/day')

    return date(int(year), month_lookup[month], int(day))

In [7]:
doc = ET.parse('../DATA/presidents.xml')
for row in doc.findall('.//president'):
    term = row.findtext('index')
    first_name = row.findtext('.//name/first')
    last_name = row.findtext('.//name/last')
    dob = get_date(doc, 'birth')
    dod = get_date(doc, 'death')
    bplace  = row.findtext('birthplace')
    bstate = row.findtext('birthstate')
    tstart = get_date(doc, 'termstart')
    tend = get_date(doc, 'termend')
    party = row.findtext('party')
    
    df.loc[term] = {   # add row to dataframe
        "first_name": first_name, "last_name": last_name, "dob": dob, "dod": dod,
        'bplace': bplace, 'bstate': bstate, 'tstart': tstart, 'tend': tend, 'party': party,
    }

df.head()

,first_name,last_name,dob,dod,bplace,bstate,tstart,tend,party
term,,,,,,,,,
1,George,Washington,1732-02-22,1799-12-14,Westmoreland County,Virginia,1789-04-30,1797-03-04,no party
2,John,Adams,1735-10-30,1826-07-04,"Braintree, Norfolk",Massachusetts,1797-03-04,1801-03-04,Federalist
3,Thomas,Jefferson,1743-04-13,1826-07-04,Albermarle County,Virginia,1801-03-04,1809-03-04,Democratic - Republican
4,James,Madison,1751-03-16,1836-06-28,Port Conway,Virginia,1809-03-04,1817-03-04,Democratic - Republican
5,James,Monroe,1758-04-28,1831-07-04,Westmoreland County,Virginia,1817-03-04,1825-03-04,Democratic - Republican


## Find first-born president for each party

Use `groupby()` to group presidents by party, then use `idxmin()` to find the index of the first entry with the minimum value. 
Then we look up the row with that index, which returns a Series, and index into the series with the column names.

In [6]:
for party, party_data in df.groupby('party'):
    min_index = party_data['dob'].idxmin() 
    president = df.loc[min_index]
    print(party, president.first_name, president.last_name, president.dob)

Democratic Andrew Jackson 1767-03-15
Democratic - Republican Thomas Jefferson 1743-04-13
Federalist John Adams 1735-10-30
Republican Andrew Johnson 1808-12-29
Whig William Henry Harrison 1773-02-09
no party George Washington 1732-02-22


## Find youngest president at start of term
It's easy to add another column to the dataframe with the age at the start of their first term by subtracting the date of birth from the term start date. That returns a `timedelta` object representing the elapsed time. That object has a `days` attribute, so we divide the days by 365.25 to get the approximate number of years. 

In [9]:
df['term_age'] = [round(d.days / 365.25, 1) for d in (df['tstart'] - df['dob'])]
index_of_youngest = df.term_age.idxmin()
df.loc[index_of_youngest]

first_name         Theodore
last_name         Roosevelt
dob              1858-10-27
dod              1919-01-06
bplace        New York City
bstate             New York
tstart           1901-09-14
tend             1909-03-04
party            Republican
term_age               42.9
Name: 26, dtype: object

In [10]:
df

,first_name,last_name,dob,dod,bplace,bstate,tstart,tend,party,term_age
term,,,,,,,,,,
1,George,Washington,1732-02-22,1799-12-14,Westmoreland County,Virginia,1789-04-30,1797-03-04,no party,57.2
2,John,Adams,1735-10-30,1826-07-04,"Braintree, Norfolk",Massachusetts,1797-03-04,1801-03-04,Federalist,61.3
3,Thomas,Jefferson,1743-04-13,1826-07-04,Albermarle County,Virginia,1801-03-04,1809-03-04,Democratic - Republican,57.9
4,James,Madison,1751-03-16,1836-06-28,Port Conway,Virginia,1809-03-04,1817-03-04,Democratic - Republican,58.0
5,James,Monroe,1758-04-28,1831-07-04,Westmoreland County,Virginia,1817-03-04,1825-03-04,Democratic - Republican,58.8
6,John Quincy,Adams,1767-07-11,1848-02-23,"Braintree, Norfolk",Massachusetts,1825-03-04,1829-03-04,Democratic - Republican,57.6
7,Andrew,Jackson,1767-03-15,1845-06-08,Waxhaw,South Carolina,1829-03-04,1837-03-04,Democratic,62.0
8,Martin,Van Buren,1782-12-05,1862-07-24,Kinderhook,New York,1837-03-04,1841-03-04,Democratic,54.2
9,William Henry,Harrison,1773-02-09,1841-04-04,Berkeley,Virginia,1841-03-04,1841-04-04,Whig,68.1


## Count presidents in each party
The `value_counts()` method will count distinct occurences of values.

In [12]:
df.value_counts('party')

party
Republican                 20
Democratic                 16
Democratic - Republican     4
Whig                        4
Federalist                  1
no party                    1
Name: count, dtype: int64

## Count presidents from each state


In [13]:
df.value_counts('bstate')

bstate
Virginia          8
Ohio              7
New York          5
Massachusetts     4
Vermont           2
Texas             2
Pennsylvania      2
North Carolina    2
New Jersey        2
New Hampshire     1
South Carolina    1
Arkansas          1
California        1
Missouri          1
Kentucky          1
Iowa              1
Illinois          1
Hawaii            1
Georgia           1
Connecticut       1
Nebraska          1
Name: count, dtype: int64